### Introduction

Udiddit, a social news aggregation, web content rating, and discussion website, is currently using a risky and unreliable Postgres database schema to store the forum posts, discussions, and votes made by their users about different topics.

The schema allows posts to be created by registered users on certain topics, and can include a URL or a text content. It also allows registered users to cast an upvote (like) or downvote (dislike) for any forum post that has been created. In addition to this, the schema also allows registered users to add comments on posts.

### Bad database schema with source data

In [ ]:
CREATE TABLE "bad_posts" (
   "id" SERIAL PRIMARY KEY,
   "topic" VARCHAR(50),
   "username" VARCHAR(50),
   "title" VARCHAR(150),
   "url" VARCHAR(4000) DEFAULT NULL,
   "text_content" TEXT DEFAULT NULL,
   "upvotes" TEXT,
   "downvotes" TEXT
);

CREATE TABLE bad_comments (
   "id" SERIAL PRIMARY KEY,
   "username" VARCHAR(50),
   "post_id" BIGINT,
   "text_content" TEXT
);


    Guideline --1: here is a list of features and specifications that Udiddit needs in order to support 
    its website and administrative interface:
    
Users:
1. Allow new users to register:
2. Each username has to be unique
3. Usernames can be composed of at most 25 characters
4. Usernames can’t be empty
5. We won’t worry about user passwords for this project

Topics:
1. Topic names have to be unique.
2. The topic’s name is at most 30 characters
3. The topic’s name can’t be empty
4. Topics can have an optional description of at most 500 characters.

Posts:
1. A comment’s text content can’t be empty.
2. Contrary to the current linear comments, the new structure should allow comment threads at arbitrary levels.
3. If a post gets deleted, all comments associated with it should be automatically deleted too.
4. If the user who created the comment gets deleted, then the comment will remain, but it will become dissociated from that user.
5. If a comment gets deleted, then all its descendants in the thread structure should be automatically deleted too.

Votes:
1. Hint: you can store the (up/down) value of the vote as the values 1 and -1 respectively.
2. If the user who cast a vote gets deleted, then all their votes will remain, but will become dissociated from the user.
3. If a post gets deleted, then all the votes for that post should be automatically deleted too.


### Creation schemas for new tables

In [ ]:
-- Topics

CREATE TABLE "topics" (
  "id" SERIAL PRIMARY KEY,
  "topic" VARCHAR(30) UNIQUE NOT NULL,
  "description" VARCHAR(500)
);
CREATE INDEX "topic_id" ON "topics" ("id");

ALTER TABLE "topics" 
ADD CONSTRAINT "not_empty" 
CHECK(LENGTH(TRIM("topic")) > 0);


In [ ]:
-- Users

CREATE TABLE "users" (
  "id" SERIAL PRIMARY KEY,
  "username" VARCHAR(25) UNIQUE NOT NULL,
  "last_login" DATE
);
CREATE INDEX "user_id" ON "users" ("id");

ALTER TABLE "users" 
ADD CONSTRAINT "not_empty" 
CHECK(LENGTH(TRIM("username")) > 0);


In [ ]:
-- Posts
--   title varchar(150) for migration, shorten to varchar(100) after
--   username and topic are temporary, delete after migration

CREATE TABLE "posts" (
  "id" SERIAL PRIMARY KEY,
  "user_id" INTEGER,
  "topic_id" INTEGER,  -- add NOT NULL after migration
  "title" VARCHAR(150) NOT NULL,
  "text_content" TEXT,
  "url" VARCHAR(4000),
  "timestamp" TIMESTAMP,
  "username" VARCHAR(50),
  "topic" VARCHAR(50)
);
CREATE INDEX "posts_id" ON "posts" ("id");

ALTER TABLE "posts" 
ADD CONSTRAINT "not_empty" 
CHECK(LENGTH(TRIM("title")) > 0);


In [ ]:
-- Comments
--   username and title are temporary, delete after migration

CREATE TABLE "comments" (
  "id" SERIAL PRIMARY KEY,
  "post_id" BIGINT,  -- add NOT NUlL after migration
  "user_id" INTEGER,
  "text_content" TEXT NOT NULL,
  "timestamp" TIMESTAMP,
  "parent_comment" INTEGER, -- add NOT NULL after migration
  "username" VARCHAR(50),
  "title" VARCHAR(150)
);
CREATE INDEX "comment_id" ON "comments" ("id");

ALTER TABLE "comments" 
ADD CONSTRAINT "not_empty" 
CHECK(LENGTH(TRIM("text_content")) > 0);


In [ ]:
-- Votes
--   title and voter are temporary, delete after migration

CREATE TABLE "votes" (
  "id" SERIAL PRIMARY KEY,
  "post_id" BIGINT,  -- add after migration
  "user_id" INTEGER,
  "upvote" INTEGER,
  "downvote" INTEGER,
  "title" VARCHAR(150),
  "voter" VARCHAR(50)
);
CREATE INDEX "voter_id" ON "votes" ("id");
CREATE INDEX "unique_vote" ON "votes" ("post_id", "user_id"); 


### Migration

In [ ]:
-- Topics

--select to confirm query
SELECT DISTINCT "topic" FROM bad_posts ORDER BY "topic";

--insertion code
INSERT INTO "topics" ("topic")
  SELECT DISTINCT "topic" 
  FROM bad_posts
  ORDER BY "topic";


In [ ]:
-- Users

--select to explore duplicates
SELECT COUNT(DISTINCT "username") 
FROM "bad_posts";             --100  usernames

SELECT COUNT(DISTINCT "username") 
FROM "bad_comments";          --9984 usernames
--note: 9984 users with comments, only 100 with posts
--duplicates split between two tables.

--create new temporary table tmp_users
--one duplicated username
CREATE TABLE "tmp_users" ("username" VARCHAR(25) NOT NULL);
INSERT INTO "tmp_users" ("username") 
  SELECT DISTINCT "username" 
  FROM "bad_comments" 
  ORDER BY "username";

INSERT INTO "tmp_users" ("username") 
  SELECT DISTINCT "username" 
  FROM "bad_posts"
  ORDER BY "username";

--check total count vs distinct count
SELECT COUNT("username") 
FROM "tmp_users";               --1084 usernames

SELECT DISTINCT "username" 
FROM "tmp_users" 
ORDER BY "username";            --1083 usernames

--insertion code for distinct users
INSERT INTO "users" ("username") 
   SELECT DISTINCT "username" 
   FROM "tmp_users" 
   ORDER BY "username";
    
-- clean up
DROP TABLE "tmp_users";


In [ ]:
-- Posts

-- select query to check
SELECT "id", "title", "text_content", "url", "username", "topic" 
FROM "bad_posts";

--insertion query   problems here, null values in topic id?
INSERT INTO "posts" 
      ("id", "title", "text_content", "url", "username", "topic")
SELECT "id", "title", "text_content", "url", "username", "topic" 
FROM "bad_posts";

--try again to get user_id. SUCCESS!
--foreign key user_id added, update constraints needed
UPDATE "posts"
SET "user_id" = (
  SELECT "id" 
  FROM "users" 
  WHERE "posts"."username" = "users"."username"
);

--repeat for topic_id
UPDATE "posts"
SET "topic_id" = (
  SELECT "id" 
  FROM "topics" 
  WHERE "posts"."topic" = "topics"."topic"
);

-- add foreign keys
ALTER TABLE "posts" 
ADD CONSTRAINT "user_id_fk" 
FOREIGN KEY ("user_id") 
REFERENCES "users" 
ON DELETE SET NULL;

ALTER TABLE "posts" 
ADD CONSTRAINT "topic_id_fk" 
FOREIGN KEY ("topic_id") 
REFERENCES "topics"
ON DELETE CASCADE;

--check constraint for url & text_content
-- ADD CONSTRAINT "not_both_url_text"
ALTER TABLE "posts" 
ADD CONSTRAINT "not_both_url_text" 
CHECK(("url" IS NOT NULL) <> ("text_content" IS NOT NULL));
--works?
INSERT INTO "posts" ("text_content", "url")
VALUES ('garbage', 'garbage');
--tested, works!

--bad_posts.title is varchar(150), too long for posts.title varchar(100)
--152 records with titles too long
SELECT COUNT(*) FROM "bad_posts" WHERE LENGTH("title") > 100;

--shorten title to 100 characters
UPDATE "posts" SET "title" = LEFT("title", 100);

--reset posts.title to varchar(100)
ALTER TABLE "posts" ALTER COLUMN "title" SET DATA TYPE varchar(100);

--drop columns username, topic
ALTER TABLE "posts" DROP COLUMN "username";
ALTER TABLE "posts" DROP COLUMN "topic";

--set FK to not null
ALTER TABLE "posts" ALTER COLUMN "topic_id" SET NOT NULL;


In [ ]:
-- Comments

INSERT INTO "comments" ("id", "post_id", "username", "text_content")
SELECT "id", "post_id", "username", "text_content" FROM "bad_comments";

UPDATE "comments" SET "user_id" = (
    SELECT "id"
    FROM "users"
    WHERE "comments"."username" = "users"."username"
  );

UPDATE "comments" SET "parent_comment" = "id";

-- drop columns "username" and "title"
ALTER TABLE "comments" DROP COLUMN "username";
ALTER TABLE "comments" DROP COLUMN "title";

-- add foreign keys: post_id, user_id, parent_comment, children_comments
ALTER TABLE "comments"
  ADD CONSTRAINT "post_id_fk"
  FOREIGN KEY ("post_id")
  REFERENCES "posts"
  ON DELETE CASCADE;

ALTER TABLE "comments"
  ADD CONSTRAINT "user_id_fk"
  FOREIGN KEY ("user_id")
  REFERENCES "users"
  ON DELETE SET NULL;

ALTER TABLE "comments"
  ADD CONSTRAINT "parent_com"
  FOREIGN KEY ("parent_comment")
  REFERENCES "comments";

ALTER TABLE "comments" ALTER COLUMN "post_id" SET NOT NULL;
ALTER TABLE "comments" ALTER COLUMN "parent_comment" SET NOT NULL;
    

    

In [ ]:
-- Votes

-- create two tmp tables: "tmp_upvote", "tmp_downvote"
-- union into "tmp_all_votes"
-- extract into "votes"


-- select statement to try regexp_split_to_table
SELECT regexp_split_to_table("upvotes",','),
       regexp_split_to_table("downvotes",','), "username"
FROM "bad_posts";

-- tmp_upvotes <- title, username (who posted), upvote_int (1 or null), username (of voter)

CREATE TABLE "tmp_upvotes" (
  "title" VARCHAR(150),
  "username" VARCHAR(50),
  "upvote" INTEGER,
  "downvote" INTEGER,
  "voter" TEXT
);


INSERT INTO "tmp_upvotes" ("title", "username", "voter")
  SELECT "title", "username", regexp_split_to_table("upvotes",',')
  FROM "bad_posts";
-- set "tmp_upvote"."upvote" = 1?
UPDATE "tmp_upvotes" SET "upvote" = 1;
-- count(*) = 249,799 rows

-- tmp_downvotes
CREATE TABLE "tmp_downvotes" (
  "title" VARCHAR(150),
  "username" VARCHAR(50),
  "upvote" INTEGER,
  "downvote" INTEGER,
  "voter" TEXT
);

INSERT INTO "tmp_downvotes" ("title", "username", "voter")
  SELECT "title", "username", regexp_split_to_table("downvotes",',')
  FROM "bad_posts";

UPDATE "tmp_downvotes" SET "downvote" = -1;
-- count(*) = 249,911 rows

-- UNION tmp_upvotes + tmp_downvotes = tmp_all_votes
CREATE TABLE "tmp_all_votes" (
  "title" VARCHAR(150),
  "username" VARCHAR(50),
  "upvote" INTEGER,
  "downvote" INTEGER,
  "voter" TEXT
);

-- select statement
SELECT * FROM "tmp_upvotes"
UNION
SELECT * FROM "tmp_downvotes";

-- insert statement
INSERT INTO "tmp_all_votes" ("title", "username", "upvote", "downvote", "voter")
  SELECT "title", "username", "upvote", "downvote", "voter"
  FROM "tmp_upvotes"
  UNION
  SELECT "title", "username", "upvote", "downvote", "voter"
  FROM "tmp_downvotes";
-- count(*) = 499,710! success!

--  data from tmp_all_votes:
INSERT INTO "votes" ("title", "upvote", "downvote", "voter")
  SELECT "title", "upvote", "downvote", "voter"
  FROM "tmp_all_votes";

-- join "votes"."voter" and "users"."username",
--   make sure all voters have "id" before inserting votes.user_id
SELECT "voter"
FROM "votes"
JOIN "users"
ON "votes"."voter" = "users"."username"
WHERE "votes"."voter" != "users"."username";
-- only 106 unique user_ids in votes table, 1100 unique usernames in votes

UPDATE "votes" SET "user_id" = (
  SELECT "id"
  FROM "users"
  WHERE "votes"."voter" = "users"."username"
);

--  count = 994 distinct voters with missing user_ids
SELECT COUNT(DISTINCT "voter")
FROM "votes"
WHERE "user_id" IS NULL;

-- select statement
SELECT DISTINCT "voter"
FROM "votes"
WHERE "user_id" IS NULL;

-- insertion into "users"
INSERT INTO "users" ("username")
  SELECT DISTINCT "voter"
  FROM "votes"
  WHERE "user_id" IS NULL;


-- shorten title from varchar(150) to varchar(100)
UPDATE "votes" SET "title" = LEFT("title", 100);

-- reset posts.title to varchar(100)
ALTER TABLE "votes" ALTER COLUMN "title" SET DATA TYPE varchar(100);


-- after initial data insertion (need title, username) populate post_id, 

UPDATE "votes" SET "user_id" = (
  SELECT "id"
  FROM "users"
  WHERE "votes"."voter" = "users"."username"
);

-- need these indexes to run update query below
CREATE INDEX "votes_title" ON "votes" ("title");
CREATE INDEX "posts_title" ON "posts" ("title");

UPDATE "votes" SET "post_id" = (
  SELECT "id"
  FROM "posts"
  WHERE "votes"."title" = "posts"."title"
);

-- add constraints after all data insertion
ALTER TABLE "votes"
  ADD CONSTRAINT "post_id_fk"
  FOREIGN KEY ("post_id")
  REFERENCES "posts"
  ON DELETE CASCADE;

ALTER TABLE "votes"
  ADD CONSTRAINT "user_id_fk"
  FOREIGN KEY ("user_id")
  REFERENCES "users"
  ON DELETE SET NULL;

ALTER TABLE "votes"
  ADD CONSTRAINT "unique_post_user"
  UNIQUE ("post_id", "user_id");

-- drop columns title, voter
ALTER TABLE "votes" DROP COLUMN "title";
ALTER TABLE "votes" DROP COLUMN "voter";

-- need to convert NULL to 0 in votes
-- UPDATE table_name SET col1 = val1, ... WHERE condition
UPDATE "votes" SET "upvote" = 0 WHERE "upvote" IS NULL;
UPDATE "votes" SET "downvote" = 0 WHERE "downvote" IS NULL;

-- clean up
DROP TABLE "tmp_upvotes", "tmp_downvotes", "tmp_all_votes";


### Queries

In [ ]:
--  a.	List all users who haven't logged in in the last year.
SELECT *
FROM  "users"
WHERE "last_login" > (CURRENT_DATE - INTERVAL '1 YEAR');


--  b.	List all users who haven't created a post.
SELECT "users"."id", "users"."username", "last_login"
FROM  "users"
LEFT JOIN "posts"  ON "users"."id" = "posts"."user_id"
WHERE "posts"."user_id" IS NULL;


--  c.	Find a user by their username.
SELECT *
FROM  "users"
WHERE "username" = ['username'];


--  d.	List all topics that don't have any posts.
SELECT *
FROM  "topics"
LEFT JOIN "posts" ON "topics"."id" = "posts"."topic_id"
WHERE "posts"."topic_id" IS NULL;


--  e.	Find a topic by its name.
SELECT *
FROM  "topics"
WHERE "topic" = ['topic_name'];


--  f.	List the latest 20 posts for a given topic.
SELECT *
FROM  "posts"
WHERE "topic" = ['topic_name']
ORDER BY "timestamp" DESC
LIMIT 20;


--  g.	List the latest 20 posts made by a given user.
SELECT *
FROM  "posts"
WHERE "user_id" = ["user_id"]
ORDER BY "timestamp" DESC
LIMIT 20;


--  h.	Find all posts that link to a specific URL, for moderation purposes.
SELECT *
FROM  "posts"
WHERE "url" = ['url'];


--  i.	List all the top-level comments (those that don't have a parent comment) for a given post.
SELECT *
FROM  "comments"
WHERE "parent_comment" IS NULL;


--  j.	List all the direct children of a parent comment.
SELECT *
FROM  "comments"
WHERE "parent_comment" = [parent INTEGER];


--  k.	List the latest 20 comments made by a given user.
SELECT *
FROM  "comments"
WHERE "user_id" = ["user_id"];


--  l.	Compute the score of a post, defined as the difference between the number 
--       of upvotes and the number of downvotes.
SELECT  
    "post_id",
    SUM("upvote") + SUM("downvote") as vote_tally
FROM "votes"
GROUP BY "post_id";